# Interpolate CROCO variables onto particles position (R_tools/Netcdf4)


- Interpolate 3d CROCO variables at particles positions
and save it to Pyticles netcdf file.

- If Pyticles outputs have a higher frequency than CROCO,
a time-linear interpolation is applied.

- Both 2D (iso-depth) and 3D Pyticles experiments are supported


To run in production mode it is advized to test your code with this notebook,
then convert it to Python script (Jupytext/jupyter-nbcvonert...)
 
___
     
 - 18/12/16:
     - add capability of computing outputs at subtime-steps period
___

In [ ]:
%matplotlib inline

Load all useful modules

In [4]:
#Plotting modules 
import matplotlib
matplotlib.use('Agg') #Choose the backend (needed for plotting inside subprocess)
import matplotlib.pyplot as plt

#Some standard modules
import sys, os
import numpy as np
import time as tm
from netCDF4 import Dataset
import multiprocessing as mp
import ctypes   

#add the Modules folder in your python PATH
sys.path.append("../Modules/")

#Specific modules needed for pyticles
import pyticles_sig_sa as part
import pyticles_3d_sig_sa as partF

#sys.path.append("/home/gula/Desktop/Work_capella/Python/Python_Modules")
#sys.path.append("/home/gula/Python_Modules")
#sys.path.append("/home2/datawork/jgula/Pyticles/Pyticles_nesea/Modules") 

#Simulations (path, data...)
from R_files import load
from R_netcdf import ionetcdf
from R_vars import var

# to avoid OSError: [Errno -101] NetCDF: HDF error:
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

In [5]:
# local functions
def linear(var1, var2, alpha):
    "Linear interpolation"
    return alpha * var2 + (1 - alpha) * var1

In [6]:
# get Pyticles file
ncfile = '/home2/datawork/jcollin/Pyticles/tutorials/polgyr_dynamic_injection_1_1000.nc'
meanflow = False

In [7]:
print('Loading file')
with Dataset(ncfile, 'r') as nc:
    parameters = nc.simulation
    base = nc.base
    ng = 0. #nc.ng # ghost points in the horizontal
    nq = len(nc.dimensions['nq'])
    ntime = len(nc.dimensions['time'])

Loading file


load simulation parameters

In [8]:
print('Loading simul')
simul = load(simul=parameters)
depths = simul.coord[4]

Loading simul
simulname is None
args (['polgyr', '[0,15000,0,15000,[1,300,1]]', '1000'],)
args[0] ['polgyr', '[0,15000,0,15000,[1,300,1]]', '1000']
len(args[0]) 3
time of simulation is: 1000
coord
loading /home/datawork-lops-osi/mlecorre/POLGYR/INIT/polgyr_grd.nc
loading /home/datawork-lops-osi/mlecorre/POLGYR/HIS/polgyr_his.01000.nc
get domain [0,15000,0,15000,[1,300,1]] [0,15
coordmax
loading /home/datawork-lops-osi/mlecorre/POLGYR/INIT/polgyr_grd.nc
loading /home/datawork-lops-osi/mlecorre/POLGYR/HIS/polgyr_his.01000.nc
get domain [0,1e9,0,1e9,[1,1e9,1]] [0,1e
cst
read Cs_r in ncfile.Cs_r
dt
dt is read in  /home/datawork-lops-osi/mlecorre/POLGYR/HIS/polgyr_his.01000.nc
[0, 1602, 0, 2002, array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,


get time

In [9]:
ocean_time = ionetcdf.get(ncfile, 'ocean_time', simul)

# old version (output generated before 16/12/20)
#time = np.round(ionetcdf.get(ncfile,'time_int',simul),2)
#time += time[2] - time[1]
# new version
ptime = np.round(ionetcdf.get(ncfile,'time',simul),3)
dtime = ptime[1] - ptime[0]
nqmx  = nq

Loop on variables

In [10]:
#for varname in ['u','v','rho1','temp','salt']:
for varname in ['rho1']:
    pvar = np.zeros(nq)
    pvarname = 'p' + varname
    
    nc = Dataset(ncfile, 'a')
    if pvarname not in list(nc.variables.keys()): 
        nc.createVariable(pvarname, 'd', ('time','nq',))
    nc.close

    ###################################################################################
    # loop on Pyticles time indices
    # alpha time manage Pyticles experiment with higher frequency outputs than CROCO
    ###################################################################################
    for filetime in ptime[:]:
        itime = int(np.abs(ptime[0] - filetime) / np.abs(dtime))
        print(itime, ptime[itime])
        if not meanflow: simul.update(np.floor(filetime));
        
        alpha_time = filetime - np.floor(filetime)
        
        ###############################################################################
        # Get positions (px,py,pz) from pyticle file
        ###############################################################################
        px = ionetcdf.get(ncfile, 'px', simul, time=itime)
        py = ionetcdf.get(ncfile, 'py', simul, time=itime)

        try:
            adv3d = False
            nc = Dataset(ncfile, 'r')
            advdepth = nc.depth
            nc.close()
        except:
            adv3d = True
            pz = ionetcdf.get(ncfile, 'pz', simul, time=itime)

        print('filetime, itime', filetime, itime)

        coord = part.subsection(px, py, ny=simul.coord[1],
                                nx=simul.coord[3], offset=10)
        tstart = tm.time()

        ######################################################################
        # Get var from simulation file
        ######################################################################
        #compute your variable here:
        if adv3d:
            myvar = var(varname, simul, coord=coord).data
        else:
            myvar = var(varname, simul, depths=[advdepth], coord=coord).data
        
        if not meanflow and alpha_time != 0:
            simul.update(np.ceil(filetime))
            if adv3d:
                myvar2 = var(varname, simul, coord=coord).data
            else:
                myvar2 = var(varname, simul, depth=[advdepth], coord=coord).data
            
            simul.update(np.floor(filetime))
            myvar = linear(myvar, myvar2, alpha_time)
            del myvar2
        
        ######################################################################
        print('get var.......................', tm.time()-tstart)
        tstart = tm.time()
        
        #if ng>0:
        #    myvar = part.periodize3d_fromvar(simul,myvar,coord,
        #               x_periodic=x_periodic,y_periodic=y_periodic,ng=ng)

        ######################################################################
        # Interpolate var to particles positions (px,py,pz)
        ######################################################################
        if adv3d:
            pvar = part.map_var(simul, myvar, px, py, pz, ng, coord=coord)
        else:
            pvar = part.map_var2d(simul, myvar, px, py, ng, coord=coord) 

        print('interpolate to particules.....', tm.time()-tstart)
        tstart = tm.time() 

        ######################################################################
        # write in file
        # for older netcdf version use 
        #  with Dataset(ncfile, 'a') as nc:
        ######################################################################
        with Dataset(ncfile, 'a', format='NETCDF4') as nc:
            nc.variables[pvarname][itime, :] = pvar
            

0 1000.0
time of simulation is: 1000
/home/datawork-lops-osi/mlecorre/POLGYR/HIS/polgyr_his.01000.nc
filetime, itime 1000.0 0
1088 1114 888 914
0 1602 0 2002
get var....................... 0.8526966571807861
interpolate to particules..... 0.00014734268188476562
1 1001.0
time of simulation is: 1001
/home/datawork-lops-osi/mlecorre/POLGYR/HIS/polgyr_his.01000.nc
filetime, itime 1001.0 1
1087 1114 887 914
0 1602 0 2002
get var....................... 0.7526721954345703
interpolate to particules..... 0.0002989768981933594
2 1002.0
time of simulation is: 1002
/home/datawork-lops-osi/mlecorre/POLGYR/HIS/polgyr_his.01000.nc
filetime, itime 1002.0 2
1085 1114 886 914
0 1602 0 2002
get var....................... 0.8190538883209229
interpolate to particules..... 0.00016999244689941406
3 1003.0
time of simulation is: 1003
/home/datawork-lops-osi/mlecorre/POLGYR/HIS/polgyr_his.01000.nc
filetime, itime 1003.0 3
1084 1114 885 914
0 1602 0 2002
get var....................... 0.8813691139221191
interpo